<a href="https://colab.research.google.com/github/gchecon/ArquivosDiversos/blob/main/SumarizacaoBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Criação de Resumos, Atualização Metadados e Carga no ElasticSearch
## Módulo de Sumarização - BERT
### O modelo treinado em português está em https://opencor.gitlab.io/corpora/souza20pre/

In [7]:
!pip install transformers
!pip install bert-extractive-summarizer
!pip install GPUtil

  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7411 sha256=f92a4dd860a4ef7fd36e1c3b137f08d6f6ec700e7577b35136e488e8aaa6f4e8
  Stored in directory: /root/.cache/pip/wheels/6e/f8/83/534c52482d6da64622ddbf72cd93c35d2ef2881b78fd08ff0c
Successfully built GPUtil


In [8]:
import torch
import os
import io
import re
from transformers import AutoConfig, AutoTokenizer, AutoModel
from summarizer import Summarizer
import gc
import time
from GPUtil import showUtilization as gpu_usage

## Leitura dos Textos

In [9]:
contentpath = './TXT/'
resumepath = './RESUMOS/'
for raiz, diretorios, arquivos in os.walk(contentpath):
    continue

In [10]:
texto_cabecalho = [] 
texto_cabecalho.append('TRIBUNAL DE CONTAS DO ESTADO DE SÃO PAULO')
texto_cabecalho.append('Rangel Pestana, 315')
texto_cabecalho.append('5º andar')
texto_cabecalho.append('CEP 01017-906')
texto_cabecalho.append('São Paulo/SP')
texto_cabecalho.append('PABX:')
texto_cabecalho.append('3292-3215')
texto_cabecalho.append('Av.')
texto_cabecalho.append('Centro')
texto_cabecalho.append(' - ')
texto_cabecalho.append('–')
def limpa_texto (str1):
    for str2 in texto_cabecalho:
        str1 = str1.replace(str2, '')
    texto = re.sub(r'\s+', ' ', str1)
    return(texto)

## Realizar a Sumarização

In [11]:
config_pt = AutoConfig.from_pretrained('neuralmind/bert-large-portuguese-cased')
config_pt.output_hidden_states=True
tokenizer_pt = AutoTokenizer.from_pretrained('neuralmind/bert-large-portuguese-cased')
model_pt = AutoModel.from_pretrained('neuralmind/bert-large-portuguese-cased', config=config_pt)

Downloading:   0%|          | 0.00/648 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/155 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/210k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at neuralmind/bert-large-portuguese-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
gc.collect()             # Para o caso do computador executar instruções em GPU Nvidia
torch.cuda.empty_cache() # Para o caso do computador executar instruções em GPU Nvidia
gpu_usage()

sumarizador = Summarizer(custom_model=model_pt, custom_tokenizer=tokenizer_pt)

t1 = time.time()
iTotal = 0

for strFile in arquivos:
    filePath = contentpath+strFile
    try:
        arquivo = io.open(filePath,'r',encoding='latin-1')
    except:
        print('Erro na Leitura do Arquivo '+strFile)
        continue
    
    texto = limpa_texto(str(arquivo.read()))
    
    if len(texto) > 1000000:
        print('Arquivo '+strFile+' possui tamanho '+str(len(texto))+' que é superior a 1000000')
    else:   
        resumo = sumarizador(texto, num_sentences=4)
    
        path = resumepath+strFile+'_res'+'.txt'
        open(path,'w+').write(resumo)
        iTotal += 1

print('Tempo de execução em segundos: ', time.time() - t1)
print('Quantidade de arquivos gerados: ', iTotal)


| ID | GPU | MEM |
------------------
|  0 |  0% | 15% |
Arquivo rtce-124.pdf.txt possui tamanho 1265232 que é superior a 1000000
Arquivo rtce-117.pdf.txt possui tamanho 1012992 que é superior a 1000000
Tempo de execução em segundos:  3353.2649097442627
Quantidade de arquivos gerados:  123


In [15]:
resumo

'2015T R IB U N A L D E C O N TA S D O E S TA D O D E S Ã O P A U LO TCESP REVISTA DO COMPOSIÇÃO DO TRIBUNAL PLENO CONSELHEIROS Reúnese s 4s feiras s 10h Cristiana de Castro Moraes Presidente Dimas Eduardo Ramalho VicePresidente Sidney Estanislau Beraldo Corregedor Antonio Roque Citadini Edgard Camargo Rodrigues Renato Martins Costa Robson Marinho CÂMARAS CONSELHEIROS REVISTA DO São Paulo Tribunal de Contas do Estado. 32 Notícias Revista do TCESP n 135 Mar. 2015 a Out. A proposta conforme consta do Plano Diretor da Reforma Administrativa do Aparelho de Estado MARE 1995 p. 15 é de realização de um controle rígido dos processos a priori como por exemplo aqueles referentes admissão de pessoal e s contratações a serem realizadas pelo Poder Públi co baseandose nos princípios da profissionaliza ção da carreira da hierarquia da impessoalidade e do formalismo. 102 Artigos Revista do TCESP n 135 Mar. 2015 a Out. Ainda tanto o e. Conselheiro Antonio Roque Citadini como o e. Conselheiro Edgard Ca